Confirm project root & paths

In [6]:
import os

os.getcwd()


'/Users/shreyayadav/ADR '

In [7]:
os.listdir()


['cadec_adr.csv',
 '.DS_Store',
 'dataset',
 'UNZIPY.ipynb',
 'error_analysis.csv',
 'PT_Extract.ipynb',
 'results.csv',
 'final_cadec_meddra_dataset.csv']

In [8]:
os.listdir("dataset")


['.DS_Store', 'data', 'metadata']

In [9]:
import zipfile

zip_path = "dataset/data/CADEC.v2.zip"
extract_path = "dataset/data/CADEC.v2"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzip complete")


Unzip complete


In [10]:
os.listdir("dataset/data")


['CADEC.v2.zip', 'CADEC.v1.zip', 'CADEC.v2']

In [11]:
os.listdir("dataset/data/CADEC.v2")


['cadec']

In [12]:
BASE_PATH = "dataset/data/CADEC.v2/cadec"




os.listdir(BASE_PATH)

['original', 'sct', 'meddra', 'text']

In [13]:
TEXT_PATH = BASE_PATH + "/text"
os.listdir(TEXT_PATH)[:5]


['LIPITOR.86.txt',
 'LIPITOR.92.txt',
 'LIPITOR.952.txt',
 'LIPITOR.946.txt',
 'LIPITOR.45.txt']

In [14]:
import os

for root, dirs, files in os.walk("dataset/data/CADEC.v2/cadec"):
    print(root)
    print("DIRS:", dirs)
    print("FILES:", files[:5])
    print("-"*40)



dataset/data/CADEC.v2/cadec
DIRS: ['original', 'sct', 'meddra', 'text']
FILES: []
----------------------------------------
dataset/data/CADEC.v2/cadec/original
DIRS: []
FILES: ['LIPITOR.95.ann', 'LIPITOR.969.ann', 'LIPITOR.81.ann', 'LIPITOR.941.ann', 'LIPITOR.799.ann']
----------------------------------------
dataset/data/CADEC.v2/cadec/sct
DIRS: []
FILES: ['LIPITOR.95.ann', 'LIPITOR.969.ann', 'LIPITOR.81.ann', 'LIPITOR.941.ann', 'LIPITOR.799.ann']
----------------------------------------
dataset/data/CADEC.v2/cadec/meddra
DIRS: []
FILES: ['LIPITOR.95.ann', 'LIPITOR.969.ann', 'LIPITOR.81.ann', 'LIPITOR.941.ann', 'LIPITOR.799.ann']
----------------------------------------
dataset/data/CADEC.v2/cadec/text
DIRS: []
FILES: ['LIPITOR.86.txt', 'LIPITOR.92.txt', 'LIPITOR.952.txt', 'LIPITOR.946.txt', 'LIPITOR.45.txt']
----------------------------------------


In [15]:
BASE_PATH = "dataset/data/CADEC.v2/cadec"
TEXT_PATH = BASE_PATH + "/text"
MEDDRA_ANN_PATH = BASE_PATH + "/meddra"

# Read a sample text file
texts = {}

for file in os.listdir(TEXT_PATH):
    if file.endswith(".txt"):
        with open(os.path.join(TEXT_PATH, file), "r", encoding="utf-8") as f:
            texts[file.replace(".txt","")] = f.read()

len(texts)


1250

In [16]:
rows = []

for file in os.listdir(MEDDRA_ANN_PATH):
    if file.endswith(".ann"):
        doc_id = file.replace(".ann","")
        with open(os.path.join(MEDDRA_ANN_PATH, file), "r", encoding="utf-8") as f:
            for line in f:
                if line.startswith("T"):
                    parts = line.strip().split("\t")

                    # 🔒 SAFETY CHECK
                    if len(parts) < 3:
                        continue   # skip broken lines

                    tag = parts[0]
                    span = parts[1]
                    text = parts[2]

                    label = span.split()[0]   # ADR
                    rows.append([doc_id, label, text])


In [17]:
import pandas as pd

df = pd.DataFrame(rows, columns=["doc_id", "label", "adr_text"])
df.head()


,doc_id,label,adr_text
0,LIPITOR.95,10011301,leg cramps
1,LIPITOR.95,10028294,cramps
2,LIPITOR.969,10033371,Severe pain in my calf muscles
3,LIPITOR.969,10033371,pain was too severe
4,LIPITOR.969,10047810,could not even walk in the morning


In [18]:
df = df[df["label"] == "ADR"].reset_index(drop=True)
print(df.shape)
df.head()


(0, 3)


,doc_id,label,adr_text


In [19]:
# full document text add karo
df["full_text"] = df["doc_id"].map(texts)

df.head()


,doc_id,label,adr_text,full_text


In [20]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text)
    return text.strip()

df["clean_adr"] = df["adr_text"].apply(clean_text)
df.head()


,doc_id,label,adr_text,full_text,clean_adr


In [21]:
print(df.shape)
df.head()



(0, 5)


,doc_id,label,adr_text,full_text,clean_adr


In [22]:
df = df[df["label"] == "ADR"].reset_index(drop=True)
df_labels = pd.DataFrame(rows, columns=["doc_id", "label", "adr_text"])
df_labels["label"].value_counts()


label
10033371             484
CONCEPT_LESS         332
10028411             278
10016256             136
10028294             129
                    ... 
10064804               1
10027296               1
10024933               1
10013781/10003071      1
10012267               1
Name: count, Length: 669, dtype: int64

In [23]:
df = pd.DataFrame(rows, columns=["doc_id", "label", "adr_text"])
print(df.shape)
df.head()


(6315, 3)


,doc_id,label,adr_text
0,LIPITOR.95,10011301,leg cramps
1,LIPITOR.95,10028294,cramps
2,LIPITOR.969,10033371,Severe pain in my calf muscles
3,LIPITOR.969,10033371,pain was too severe
4,LIPITOR.969,10047810,could not even walk in the morning


Cell 9: Train–Test Split
Purpose: Evaluation-ready split

In [24]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42
)

print(train_df.shape, test_df.shape)


(5052, 3) (1263, 3)


Cell 10: TF–IDF Baseline Model

Purpose: Classical NLP baseline

In [39]:
import re

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-z\s]", "", text)
    return text.strip()

train_df["clean_adr"] = train_df["adr_text"].apply(clean_text)
test_df["clean_adr"]  = test_df["adr_text"].apply(clean_text)

train_df[["adr_text", "clean_adr"]].head()



,adr_text,clean_adr
0,head aches,head aches
1,cancer,cancer
2,CPK mildly elevated,cpk mildly elevated
3,dizziness,dizziness
4,charley horses in breasts,charley horses in breasts


In [40]:
print(train_df.columns)


Index(['doc_id', 'label', 'adr_text', 'clean_adr'], dtype='object')


Cell 11: Sentence-BERT Embeddings

Purpose: Semantic matching (main contribution)

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()

X_train = tfidf.fit_transform(train_df["clean_adr"])
X_test  = tfidf.transform(test_df["clean_adr"])

sim = cosine_similarity(X_test, X_train)

sim.shape


(1263, 5052)

In [42]:
print("baseline_acc exists?", "baseline_acc" in globals())
print("bert_acc exists?", "bert_acc" in globals())


baseline_acc exists? False
bert_acc exists? True


In [43]:
train_df = train_df.reset_index(drop=True)
test_df  = test_df.reset_index(drop=True)


In [44]:
train_df["clean_adr"] = train_df["clean_adr"].astype(str)
test_df["clean_adr"]  = test_df["clean_adr"].astype(str)


In [47]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer("all-MiniLM-L6-v2")

train_emb = model.encode(
    train_df["clean_adr"].tolist(),
    show_progress_bar=True
)

test_emb = model.encode(
    test_df["clean_adr"].tolist(),
    show_progress_bar=True
)

sim_bert = cosine_similarity(test_emb, train_emb)

pred_idx = np.argmax(sim_bert, axis=1)
bert_preds = train_df.iloc[pred_idx]["adr_text"].values

bert_acc = (bert_preds == test_df["adr_text"].values).mean()
print("Sentence-BERT Accuracy:", bert_acc)


Batches:   0%|          | 0/158 [00:00<?, ?it/s]

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

Sentence-BERT Accuracy: 0.4172604908946952


/opt/miniconda3/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/opt/miniconda3/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/opt/miniconda3/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


Sentence-BERT ke liye Accuracy@1

In [57]:
def accuracy_at_1(sim, train_df, test_df):
    top1_idx = sim.argmax(axis=1)
    correct = 0

    for i in range(len(test_df)):
        true = test_df.iloc[i]["adr_text"].lower()
        pred = train_df.iloc[top1_idx[i]]["adr_text"].lower()
        if true == pred:
            correct += 1

    return correct / len(test_df)


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()

X_train = tfidf.fit_transform(train_df["clean_adr"])
X_test  = tfidf.transform(test_df["clean_adr"])

sim_tfidf = cosine_similarity(X_test, X_train)


In [60]:
baseline_acc = accuracy_at_1(sim_tfidf, train_df, test_df)
baseline_acc


0.5376088677751386

In [61]:
results = pd.DataFrame({
    "Model": ["TF-IDF", "Sentence-BERT"],
    "Accuracy@1": [baseline_acc, bert_acc]
})

results


,Model,Accuracy@1
0,TF-IDF,0.537609
1,Sentence-BERT,0.417260


In [62]:
df.to_csv("final_cadec_meddra_dataset.csv", index=False)
results.to_csv("results.csv", index=False)


In [63]:
import numpy as np

# Top-5 indices for each test sample
top5_idx = np.argsort(sim_bert, axis=1)[:, -5:]


In [64]:
top5_preds = []

for i in range(len(top5_idx)):
    preds = train_df.iloc[top5_idx[i]]["adr_text"].values
    top5_preds.append(preds)


In [65]:
correct = 0

for i, true_adr in enumerate(test_df["adr_text"].values):
    if true_adr in top5_preds[i]:
        correct += 1

acc_at_5 = correct / len(test_df)
print("Sentence-BERT Accuracy@5:", acc_at_5)


Sentence-BERT Accuracy@5: 0.47901821060965954


In [66]:
results = pd.DataFrame({
    "Model": ["TF-IDF", "Sentence-BERT"],
    "Accuracy@1": [baseline_acc, bert_acc],
    "Accuracy@5": [None, acc_at_5]
})

results


,Model,Accuracy@1,Accuracy@5
0,TF-IDF,0.537609,NaN
1,Sentence-BERT,0.417260,0.479018


ERROR ANALAYSIS

In [67]:
error_rows = []

for i in range(len(test_df)):
    true_adr = test_df.iloc[i]["adr_text"]
    pred_adr = bert_preds[i]

    if true_adr != pred_adr:
        error_rows.append([
            test_df.iloc[i]["clean_adr"],
            true_adr,
            pred_adr,
            top5_preds[i]
        ])

error_df = pd.DataFrame(
    error_rows,
    columns=["Input_ADR", "True_ADR", "Predicted_ADR", "Top5_Predictions"]
)

error_df.head()


,Input_ADR,True_ADR,Predicted_ADR,Top5_Predictions
0,blured vision,Blured vision,blured vision,"[blurring vision, blurry vision, blurry vision..."
1,stiff elbows,stiff elbows,Arm stiffness,"[Joint Stiffness, joint stiffness, stiff joint..."
2,vertigo,VERTIGO,vertigo,"[vertigo, vertigo, vertigo, vertigo, vertigo]"
3,chest pains,Chest pains,chest pains,"[intense chest pains, chest pains, chest pains..."
4,turning in bed is painful,turning in bed is painful,moving in bed is painful,"[unable to get out of bed, hurts to get up in ..."


In [68]:
error_df["Error_Type"] = error_df.apply(
    lambda x: "Lexical variation"
    if x["True_ADR"] in x["Top5_Predictions"]
    else "Semantic confusion",
    axis=1
)

error_df["Error_Type"].value_counts()


Error_Type
Semantic confusion    655
Lexical variation      81
Name: count, dtype: int64

In [69]:
error_df.to_csv("error_analysis.csv", index=False)


In [70]:
import numpy as np

def accuracy_at_k(similarity_matrix, train_df, test_df, k=5):
    """
    similarity_matrix: (n_test, n_train)
    k: top-k
    """
    topk_idx = np.argsort(similarity_matrix, axis=1)[:, -k:]
    
    correct = 0
    for i in range(len(test_df)):
        true_adr = test_df.iloc[i]["adr_text"]
        topk_preds = train_df.iloc[topk_idx[i]]["adr_text"].values
        
        if true_adr in topk_preds:
            correct += 1
    
    return correct / len(test_df)


In [71]:
acc_at_1 = accuracy_at_k(sim_bert, train_df, test_df, k=1)
acc_at_3 = accuracy_at_k(sim_bert, train_df, test_df, k=3)
acc_at_5 = accuracy_at_k(sim_bert, train_df, test_df, k=5)

print("A@1:", acc_at_1)
print("A@3:", acc_at_3)
print("A@5:", acc_at_5)


A@1: 0.39667458432304037
A@3: 0.4750593824228028
A@5: 0.47901821060965954


In [72]:
acc_tfidf_1 = accuracy_at_k(sim, train_df, test_df, k=1)
acc_tfidf_5 = accuracy_at_k(sim, train_df, test_df, k=5)

print("TF-IDF A@1:", acc_tfidf_1)
print("TF-IDF A@5:", acc_tfidf_5)


TF-IDF A@1: 0.4338875692794933
TF-IDF A@5: 0.4845605700712589


In [73]:
results = pd.DataFrame({
    "Model": ["TF-IDF", "Sentence-BERT"],
    "A@1": [acc_tfidf_1, acc_at_1],
    "A@3": [None, acc_at_3],
    "A@5": [acc_tfidf_5, acc_at_5]
})

results


,Model,A@1,A@3,A@5
0,TF-IDF,0.433888,NaN,0.484561
1,Sentence-BERT,0.396675,0.475059,0.479018


In [74]:
import pandas as pd

# rows already bana hua list hai: [doc_id, label, adr_text]
df = pd.DataFrame(rows, columns=["doc_id", "label", "adr_text"])

print("DataFrame shape:", df.shape)
df.head()


DataFrame shape: (6315, 3)


,doc_id,label,adr_text
0,LIPITOR.95,10011301,leg cramps
1,LIPITOR.95,10028294,cramps
2,LIPITOR.969,10033371,Severe pain in my calf muscles
3,LIPITOR.969,10033371,pain was too severe
4,LIPITOR.969,10047810,could not even walk in the morning
